In [ ]:
import os

import numpy as np
import pandas as pd

evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals-2024-04-20')
predictions_file = os.path.join(evaluation_dir, 'predictions.csv')

predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['execution_time'] = pd.to_timedelta(predictions['execution_time'])

ground_truth_file = os.path.join(evaluation_dir, 'ground_truth.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions.head()

## Compare the signal with the ground truth

In [ ]:
from scipy.signal import detrend

In [ ]:
test_subject = 'Proband16'

scenario = predictions[
    (predictions['subject'] == test_subject) &
    (predictions['scenario'] == '101_natural_lighting') &
    (predictions['method'] == 'mtts_can') &
    (predictions['roi'] == 'full')]
    # (predictions['method'] == 'optical_flow') &
    # (predictions['roi'] == 'chest')]


prediction_signal = scenario.iloc[0]['signal']
sampling_rate = scenario.iloc[0]['sampling_rate']

In [ ]:
from scipy.signal import resample

ground_truth_signal = ground_truth[
    (ground_truth['subject'] == test_subject) &
    (ground_truth['scenario'] == '101_natural_lighting')
    ].iloc[0]

# Down sample the signal to predict the frequency
ground_truth_signal = resample(ground_truth_signal['signal'], len(prediction_signal))

gt_detrend = detrend(ground_truth_signal)

In [ ]:
import matplotlib.pyplot as plt

_, axs = plt.subplots(2, 1, figsize=(20, 10))

# Plot the ground truth signal and the predicted signal
axs[0].set_title('Ground Truth')
axs[0].plot(ground_truth_signal, label='Ground Truth')
axs[0].grid(True)

axs[1].set_title('Ground Truth (Detrend)')
axs[1].plot(gt_detrend, label='Ground Truth (Detrend)')
axs[1].grid(True)

In [ ]:
prediction_detrend = detrend(prediction_signal)

In [ ]:
import matplotlib.pyplot as plt

_, axs = plt.subplots(2, 1, figsize=(20, 10))

# Plot the ground truth signal and the predicted signal
axs[0].set_title('Predection')
axs[0].plot(prediction_signal, label='Predection')
axs[0].grid(True)

axs[1].set_title('Predection (Detrend)')
axs[1].plot(prediction_detrend, label='Predection')
axs[1].grid(True)

In [ ]:
import respiration.preprocessing as sp

gt_normalized = sp.normalize_signal(gt_detrend)
pd_normalized = sp.normalize_signal(prediction_detrend)

In [ ]:
import matplotlib.pyplot as plt

_, axs = plt.subplots(3, 1, figsize=(20, 10))

# Plot the ground truth signal and the predicted signal
axs[0].set_title('Ground Truth')
axs[0].plot(gt_normalized, label='Ground Truth')
axs[0].grid(True)

axs[1].set_title('Predicted')
axs[1].plot(pd_normalized, label='Predicted')
axs[1].grid(True)

# Plot the ground truth signal and the predicted signal
axs[2].set_title('Diff')
axs[2].plot(pd_normalized - gt_normalized, label='Diff')
# axs[2].plot(gt_normalized, label='Ground Truth')
axs[2].grid(True)

In [ ]:
gt_filtered = sp.butterworth_filter(gt_normalized, sampling_rate, 0.08, 0.6)
pd_filtered = sp.butterworth_filter(pd_normalized, sampling_rate, 0.08, 0.6)

gt_filtered = sp.normalize_signal(gt_filtered)
pd_filtered = sp.normalize_signal(pd_filtered)

In [ ]:
import matplotlib.pyplot as plt

_, axs = plt.subplots(3, 1, figsize=(20, 10))

axs[0].set_title('Ground Truth (filtered)')
axs[0].plot(gt_filtered, label='Ground Truth')
axs[0].grid(True)

axs[1].set_title('Predicted (filtered)')
axs[1].plot(pd_filtered, label='Predicted')
axs[1].grid(True)

# Plot the ground truth signal and the predicted signal
axs[2].set_title('Filtered signals')
axs[2].plot(pd_filtered, label='Prediction')
axs[2].plot(gt_filtered, label='Ground Truth')
axs[2].grid(True)
axs[2].legend()

In [ ]:
import respiration.analysis as analysis

dist_normalized = analysis.distance_euclidean(pd_normalized, gt_normalized)
dist_filtered = analysis.distance_euclidean(pd_filtered, gt_filtered)

print(f'dist_normalized: {dist_normalized:.2f}')
print(f'dist_filtered:   {dist_filtered:.2f}')

In [ ]:
mse_normalized = analysis.distance_mse(pd_normalized, gt_normalized)
mse_filtered = analysis.distance_mse(pd_filtered, gt_filtered)

print(f'mse_normalized: {mse_normalized:.2f}')
print(f'mse_filtered:   {mse_filtered:.2f}')

In [ ]:
normalized_correlation = analysis.pearson_correlation(pd_normalized, gt_normalized)
filtered_correlation = analysis.pearson_correlation(pd_filtered, gt_filtered)

print(f'Normalized: {normalized_correlation:.2f}')
print(f'Filtered:   {filtered_correlation:.2f}')